In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReviewAnalysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:

from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
filtered_games_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_games_df.show(5)

In [ ]:
# The data is filtered to create a DataFrame where the total_votes count is equal to or greater than 20 to pick reviews
filtered_twenty_df = filtered_games_df.filter(filtered_games_df['total_votes'] >= 20)
filtered_twenty_df.show(5)

In [ ]:
# Create the DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
fifty_percent_df = filtered_twenty_df.filter(filtered_twenty_df["helpful_votes"]/filtered_twenty_df["total_votes"] >= 0.5)
fifty_percent_df.show(5)

In [ ]:
# create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_review_df = fifty_percent_df.filter(fifty_percent_df['vine'] == 'Y')
vine_review_df.show(5)

In [ ]:
# retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
no_vine_review_df = fifty_percent_df.filter(fifty_percent_df['vine'] == 'N')
no_vine_review_df.show(5)

In [ ]:
#result questions vine amounts
total_vine = vine_review_df.count()
print(total_vine)

In [ ]:
vine_five_star_amount = vine_review_df.filter(vine_review_df['star_rating'] == 5).count()
print(vine_five_star_amount)

In [ ]:
percentage_five_star_vine = float(vine_five_star_amount) / float(total_vine)
print(percentage_five_star_vine)

In [ ]:
# result questions non vine amounts
no_vine = no_vine_review_df.count()
print(no_vine)

In [ ]:
no_vine_five_star_amount = no_vine_review_df.filter(vine_review_df['star_rating'] == 5).count()
print(no_vine_five_star_amount)

In [ ]:
percentage_five_star_no_vine = float(no_vine_five_star_amount) / float(no_vine)
print(percentage_five_star_no_vine)

In [ ]:
no_vine + total_vine